### Using Qiskit's feature maps to test out banking dataset 

In [8]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC                             # Library to implementing SVM
from sklearn.inspection import DecisionBoundaryDisplay  # Library to visualize the boundary

# Qiskit Libraries for Feature Maps and the Quantum Kernel
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay, f1_score
from sklearn.preprocessing import StandardScaler

seed = 23743                                               
algorithm_globals.random_seed = seed

In [14]:
df = pd.read_csv('bank_cleaned.csv')

bank_data = df.sample(n=20)

y = bank_data['y'].values
X = bank_data[['age','job','marital','education','default','balance',
               'housing','loan','contact','day','month','duration',
               'campaign','pdays','previous','poutcome']].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=12)
ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)


In [26]:
# Defining our quantum feature map
feature_map = ZZFeatureMap(feature_dimension=16, reps=2) 

# Defining the backend we are going to use for our kernel
backend = QuantumInstance(BasicAer.get_backend("qasm_simulator"), 
                          shots=1, seed_simulator=seed, seed_transpiler=seed) 

In [27]:
# Definig QuantumKernel
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# Using SVC from sklearn 
svc = SVC(kernel=kernel.evaluate)
svc.fit(X_train, y_train)

# Testing our model with a inbuilt score method
score = svc.score(X_test,y_test)

print(f"Classification test score: {score}")

Classification test score: 0.5


In [28]:
svc.predict(X_test)

array([0, 0], dtype=int64)